# Working with our SQL Database

## Prior to running this file you must have run the "Cleaning_Data.ipynb" file and have imported the two exported csv files into a database on your local pgadmin server

In [1]:
# Importing library
import pandas as pd
import psycopg2
import csv

# Connecting to db
con = psycopg2.connect(
    host = "localhost",
    database = "<DATABASE NAME>", # USE YOUR OWN DATABASE NAME HERE
    user= "<USERNAME>", # USE YOUR OWN USER NAME HERE
    password= "<PASSWORD>", # USE YOUR OWN PASSWORD HERE
    port= 5432 # USE YOUR OWN PORT HERE - DEFAULT IS 5432
)

# Activate connection cursor
cur = con.cursor()

# Executing query
sql_query = '''SELECT \
    DISTINCT(cc.county), \
    cc.state, \
    cc.date, \
    cc.fips, \
    cc.cases, \
    cc.deaths, \
    cc.stay_at_home_announced, \
    cc.stay_at_home_effective, \
    cc.latitude, \
    cc.longitude, \
    cc.total_population, \
    cc.percent_single_parent_households_CHR, \
    cc.percent_adults_with_diabetes, \
    cc.percent_food_insecure, \
    cc.percent_insufficient_sleep, \
    cc.percent_unemployed_CDC, \
    cc.percent_no_highschool_diploma, \
    cc.percent_age_65_and_older, \
    cc.percent_age_17_and_younger, \
    cc.percent_disabled, \
    cc.percent_multi_unit_housing, \
    cc.percent_fair_or_poor_health, \
    cc.percent_not_proficient_in_english, \
    cc.percent_female, \
    cc.percent_rural, \
    cc.percent_minorities, \
    state_winner.winning_party \
FROM covid_counties AS cc \
LEFT JOIN \
    (SELECT state_votes.state, \
        CASE WHEN republican_votes > democrat_votes \
        THEN 'REP' \
        ELSE 'DEM' END AS "winning_party" \
    FROM( \
        SELECT rep.state, \
            republican_votes, \
            democrat_votes \
        FROM (SELECT state, \
            SUM(total_votes) as republican_votes \
            FROM election_results \
            WHERE party = 'REP' \
            GROUP BY state) as rep \
        JOIN \
            (SELECT state, \
            SUM(total_votes) as democrat_votes \
            FROM election_results \
            WHERE party = 'DEM' \
            GROUP BY state) AS dem \
        ON rep.state = dem.state \
        ORDER BY rep.state) AS state_votes) AS state_winner \
ON cc.state = state_winner.state \
WHERE cc.date = '2020-11-03' \
ORDER BY county;''' 

# Execute the query
cur.execute(sql_query)

# Get the results
results = cur.fetchall()

# Converting the results into a dataframe
df = pd.DataFrame(results)

# Giving the dataframe column headers
df.columns = [
            "county",
            "state",
            "date",
            "fips",
            "cases",
            "deaths",
            "stay_at_home_announced",
            "stay_at_home_effective",
            "latitude",
            "longitude",
            "total_population",
            "percent_single_parent_households_CHR",
            "percent_adults_with_diabetes",
            "percent_food_insecure", 
            "percent_insufficient_sleep", 
            "percent_unemployed_CDC", 
            "percent_no_highschool_diploma", 
            "percent_age_65_and_older", 
            "percent_age_17_and_younger", 
            "percent_disabled",
            "percent_multi_unit_housing", 
            "percent_fair_or_poor_health", 
            "percent_not_proficient_in_english", 
            "percent_female", 
            "percent_rural",
            "percent_minorities",
            "winning_party"
             ]

# Exporting the dataframe as a CSV - YOUR PATH WILL BE DIFFERENT
df.to_csv(r"<CUSTOM PATH>\final_database.csv", index=False)
        
# Closing the cursor -- IMPORTANT --
cur.close()
del cur

# Closing the connection -- IMPORTANT -- 
con.close()